# EN.705.625 — Introduction to Agentic AI  
## Module 11 Review Questions  

**Instructions:**  
Answer each question by completing the **Python code cell below** it.  
Include comments in your code to explain your reasoning and design choices.

### 1. Planner Agent

Build a function `planner_agent(goal)` that:

1. Generates a **5-step plan** to achieve the given goal.  
2. Executes each step sequentially.  
3. Stores each **outcome** in memory.  
4. Returns a **summary** of what was achieved.


In [1]:
import ollama

def ollama_chat(prompt, model="qwen2.5:32b"):
    """
    Helper function to send a single-turn query to a local Ollama model.
    """
    response = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    return response["message"]["content"].strip()


def generate_plan(goal, model="qwen2.5:32b"):
    """
    Produces a 5-step actionable plan.
    """
    prompt = f"""
    You are a planning assistant.
    Create a clear 5-step plan to accomplish the goal below.
    Keep steps concrete and imperative.

    Goal: {goal}

    Format:
    1. Step one
    2. Step two
    ...
    """
    raw = ollama_chat(prompt, model)
    steps = [
        line.split(".", 1)[1].strip()
        for line in raw.splitlines()
        if line.strip().startswith(tuple(str(i) for i in range(1,10)))
    ]
    return steps[:5]


def execute_step(step, model="qwen2.5:32b"):
    """
    Executes a step by having the model describe/perform it.
    """
    prompt = f"""
    Execute the following step and explain what was done:
    Step: "{step}"
    Provide the result.
    """
    return ollama_chat(prompt, model)


def planner_agent(goal, model="qwen2.5:32b"):
    """
    - Generates plan
    - Executes steps sequentially
    - Stores outcomes in memory
    - Produces final summary
    """
    steps = generate_plan(goal, model)
    memory = []

    print("\n=== PLAN ===")
    for i, step in enumerate(steps, start=1):
        print(f"{i}. {step}")

    print("\n=== EXECUTION ===")
    for step in steps:
        outcome = execute_step(step, model)
        print(f"→ Result: {outcome}\n")
        memory.append((step, outcome))

    # Summarize final progress
    summary_prompt = "Summarize what was achieved:\n\n"
    for step, outcome in memory:
        summary_prompt += f"- {step}: {outcome}\n"
    summary = ollama_chat(summary_prompt, model)

    return {
        "plan": steps,
        "memory": memory,
        "summary": summary
    }


# Example usage
result = planner_agent("Learn how to grow strawberries indoors", model="qwen2.5:32b")
print("\n=== SUMMARY ===")
print(result["summary"])



=== PLAN ===
1. Select a suitable variety of strawberry that can thrive in indoor conditions, such as 'Albion' or 'Seascape'.
2. Choose and prepare the right container with adequate drainage holes for your strawberries, ensuring it’s at least 6-8 inches deep.
3. Fill the containers with high-quality potting mix designed for indoor plants, mixing in some compost to improve soil fertility.
4. Plant strawberry seedlings or runners into the pots, placing them about 12 inches apart to ensure good air circulation and growth space.
5. Position your potted strawberries near a sunny window or under grow lights, maintaining temperatures between 60-80°F during the day and not lower than 50°F at night for optimal growth conditions.

=== EXECUTION ===
→ Result: Step Execution:

To fulfill this step, we first need to understand what is required: selecting a type of strawberry plant that is well-suited for growing indoors. Among many varieties, two specific ones were mentioned as examples: 'Albion' 

### 2. Persistent Reflective Agent

Tie everything together into a single **agent class** that:

1. Maintains **short-term** and **long-term** memory.  
2. Can **summarize** interactions for efficient recall.  
3. Can **retrieve** and **reflect** on past experiences.  
4. Periodically **prunes** or compresses memory to maintain relevance.


In [ ]:
import ollama
from datetime import datetime

def ollama_chat(prompt, model="qwen2.5:32b"):
    response = ollama.chat(
        model=model,
        messages=[{"role": "user", "content": prompt}]
    )
    return response["message"]["content"].strip()


class ReflectiveAgent:
    def __init__(self, model="qwen2.5:32b", short_term_limit=10):
        self.model = model
        self.short_term_limit = short_term_limit
        
        self.short_term_memory = []  # recent detailed experiences
        self.long_term_memory = []   # compressed knowledge summaries

    # ===== Memory Core =====

    def remember(self, event: str):
        """Store raw event in short-term memory, prune if needed."""
        self.short_term_memory.append((datetime.utcnow(), event))
        if len(self.short_term_memory) > self.short_term_limit:
            self._consolidate_memory()

    def _consolidate_memory(self):
        """Compress short-term memory into long-term knowledge."""
        text = "\n".join([e for _, e in self.short_term_memory])
        prompt = f"""
        Summarize the following agent experiences into stable, reusable knowledge.
        Focus on skills learned and patterns identified.

        Experiences:
        {text}

        Produce 3-5 bullet points representing distilled insights.
        """
        summary = ollama_chat(prompt, self.model)
        self.long_term_memory.append((datetime.utcnow(), summary))
        self.short_term_memory = []

    def retrieve_relevant(self, query: str):
        """Retrieve relevant memory from both memory banks."""
        all_mem = "\n".join(
            [f"[ST] {e}" for _, e in self.short_term_memory] +
            [f"[LT] {s}" for _, s in self.long_term_memory]
        )
        prompt = f"""
        The user asked: "{query}"

        Given the memory below, select the details that are relevant.
        If none are relevant, say "No relevant memory."

        Memory:
        {all_mem}
        """
        return ollama_chat(prompt, self.model)

    def reflect(self):
        """Periodically generate insights."""
        if not self.long_term_memory:
            return "No long-term memory yet."
        text = "\n".join([s for _, s in self.long_term_memory[-3:]])
        prompt = f"""
        Reflect on the agent’s recent knowledge summaries below.
        Identify emerging themes, strengths, and growth direction.

        Knowledge:
        {text}

        Output a concise reflection paragraph.
        """
        return ollama_chat(prompt, self.model)

    # ===== Action Interface =====

    def plan_and_execute(self, goal: str):
        """Your previous planner_agent, now memory-aware."""
        plan_prompt = f"""
        Create a clear 5-step plan to achieve the goal:
        {goal}
        """
        plan = ollama_chat(plan_prompt, self.model).splitlines()

        # clean plan lines
        steps = [
            line.split(".", 1)[1].strip()
            for line in plan
            if line.strip().startswith(tuple(str(i) for i in range(1,10)))
        ][:5]

        results = []
        for step in steps:
            result = ollama_chat(f"Execute and explain: {step}", self.model)
            results.append((step, result))
            self.remember(f"STEP: {step}\nRESULT: {result}")

        # Final summary
        summary_text = "\n".join([f"{s}: {r}" for s, r in results])
        final_summary = ollama_chat(f"Summarize overall progress:\n{summary_text}", self.model)

        return {
            "plan": steps,
            "results": results,
            "summary": final_summary,
            "reflection": self.reflect()
        }


### 3. Prompt-First Tool Router

Design a **prompt** that:

1. Classifies a user’s task into one of three categories: *math*, *text*, or *lookup*.  
2. Proposes a **single plan step** based on that classification.  
3. Emits a **JSON tool call** with the chosen tool and any necessary arguments.  
4. Asks **one clarifying question** if uncertain about the task type.

Use the following schema:

```json
{
  "task_type": "math",
  "plan": "Perform the multiplication to compute the result.",
  "tool_call": {"tool": "math_solver", "args": {"expression": "23 * 47"}},
  "clarification": ""
}
```


"""
You are a tool-routing reasoning module.  
Given the user's request, decide which type of task it is, propose a plan, and format your response in JSON only.

You must classify the task into exactly one of these categories:

1. "math" — involves computation, equations, symbolic manipulation, numeric operations.
2. "text" — involves writing, editing, summarizing, translating, or generating natural language.
3. "lookup" — requires retrieving factual or external information from a database, API, or search tool.

If you are **uncertain** which category applies, ask **one clarifying question** and do NOT commit to a final tool call yet. Set "tool_call" to null when asking clarification.

When you are confident, propose exactly **one next-step plan action** and produce a **JSON tool call**:

Tools available:
- math_solver(expression)
- text_writer(instructions)
- lookup_query(query)

Output must follow this schema exactly:

{
  "task_type": "",
  "plan": "",
  "tool_call": {"tool": "", "args": {}},
  "clarification": ""
}

Behavior Rules:
- Do not speak outside JSON.
- If clarification is needed, "task_type", "plan", and "tool_call" should be empty strings or null.
- Always keep arguments minimal and directly relevant.
- Never output code fences around your JSON.
"""
